In [ ]:
import numpy as np
from imutils import contours
import cv2


image = cv2.imread(r"C:\Users\User\Desktop\New Votes DataSet\New Votes DataSet\Invalid Votes\1733843944290.jpg")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (5, 5), 0)
thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]


cnts = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
for c in cnts:
    area = cv2.contourArea(c)
    if area <50:
        cv2.drawContours(thresh, [c], -1, 0, -1)


invert = 255 - thresh
offset, old_cY, old_cX, first = 10, 0, 0, True
visualize = cv2.cvtColor(invert, cv2.COLOR_GRAY2BGR)


cnts = cv2.findContours(invert, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
(cnts, _) = contours.sort_contours(cnts, method="top-to-bottom")
crosscount=0
table = []
row = []
rownumber=0
current_row_index = -1 
current_column_index = -1
count=0
excluded_row_y_range = (600,900)
column_centroids = []
values = []
x_marks = []
processed_centroids = []  
black_pixel_counter = 0
for c in cnts:
    M = cv2.moments(c)
    if M["m00"] > 0:
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])

        if 0 <= cX <= 1000:
            continue
            
        too_close = False
        for (prev_cX, prev_cY) in processed_centroids:
            if abs(cX - prev_cX) < 200 and abs(cY - prev_cY) < 200: 
                too_close = True
                break

        if too_close:
            continue  


        processed_centroids.append((cX, cY))


        if abs(cY - old_cY) > offset:
            if not first:
                table.append(row)
            old_cY = cY
            row = []
            current_row_index += 1

        column_index = -1
        for i, column_cX in enumerate(column_centroids):
            if abs(cX - column_cX) < 50:
                column_index = i
                break

        if column_index == -1:
            column_index = len(column_centroids)
            column_centroids.append(cX)
            
        if column_index ==0:
            for inner_c in cnts:
#                 if crosscount>2:
#                     break
                inner_M = cv2.moments(inner_c)
                if inner_M["m00"] > 0:
                    inner_cX = int(inner_M["m10"] / inner_M["m00"])
                    inner_cY = int(inner_M["m01"] / inner_M["m00"])
                    if abs(inner_cX - cX) < 50:  
                        inner_epsilon = 0.04 * cv2.arcLength(inner_c, True)
                        inner_approx = cv2.approxPolyDP(inner_c, inner_epsilon, True)
                        print(len(inner_approx) )
                        if len(inner_approx) == 8:
                            crosscount=crosscount+1
                            x, y, w, h = cv2.boundingRect(inner_approx)
                            roi =visualize[y:y+h, x:x+w]
                            if np.any(roi == 0):
                                black_pixel_counter=black_pixel_counter+1
                                cv2.rectangle(visualize, (x, y), (x + w, y + h), (0, 255, 0), 2)
                            

            for inner_c in cnts:
                if count>=1:
                    break
                inner_M = cv2.moments(inner_c)
                if inner_M["m00"] > 0:
                    inner_cX = int(inner_M["m10"] / inner_M["m00"])
                    inner_cY = int(inner_M["m01"] / inner_M["m00"])
                    if abs(inner_cX - cX) < 50:  
                        inner_epsilon = 0.04 * cv2.arcLength(inner_c, True)
                        inner_approx = cv2.approxPolyDP(inner_c, inner_epsilon, True)
                        rownumber=rownumber+1
#                         print(len(inner_approx))
                        if len(inner_approx) == 8:
                            count=count+1
                            ix, iy, iw, ih = cv2.boundingRect(inner_approx)
                            cv2.rectangle(visualize, (ix, iy), (ix + iw, iy + ih), (255, 0, 0), 2)
                            cv2.putText(
                                visualize,
                                f"Detected: ({rownumber-2}, {column_index})",
                                (inner_cX - 50, inner_cY - 10),
                                cv2.FONT_HERSHEY_SIMPLEX,
                                0.8,
                                (0, 255,0),
                                2,
                            )

print(black_pixel_counter)
scale_percent = 40 
width = int(visualize.shape[1] * scale_percent / 100)
height = int(visualize.shape[0] * scale_percent / 100)
dim = (width, height)


resized_visualize = cv2.resize(visualize, dim, interpolation=cv2.INTER_AREA)


cv2.imshow('Visualize', resized_visualize)
cv2.waitKey(0)
cv2.destroyAllWindows()


4
8
4
4
7
4
6
6
4
8
4
4
7
4
6
6
4
8
4
4
7
4
6
6
4
8
4
4
7
4
6
6
4
8
4
4
7
4
6
6
4
8
4
4
7
4
6
6
6


In [ ]:
pip install imutils

In [ ]:
pip install pytesseract